In [12]:
import importlib
importlib.reload(ca_model)

<module 'ca_model' from 'drive/MyDrive/Code/GitHub/CellularAutomata/ca_model.py'>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
folder = 'drive/MyDrive/Code/GitHub/CellularAutomata'
import sys
sys.path.insert(1, folder)
import ca_model
import count_pixels_dataset as cpd

In [3]:
import numpy as np
import matplotlib.pyplot as plt
# import tensorflow as tf
import matplotlib.colors as cl
# from tensorflow.keras.layers import Conv2D
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
import moviepy.editor as mvp
import platform, tqdm
import PIL.Image, PIL.ImageDraw
from IPython.display import Image, clear_output
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3719168/45929032 bytes (8.1%)7970816/45929032 bytes (17.4%)12263424/45929032 bytes (26.7%)16629760/45929032 bytes (36.2%)20881408/45929032 bytes (45.5%)25083904/45929032 bytes (54.6%)29245440/45929032 bytes (63.7%)33579008/45929032 bytes (73.1%)37691392/45929032 bytes (82.1%)41877504/45929032 bytes (91.2%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root/.imageio/ffmpeg/ffmpeg-linux64-v3.3.1.
[name: "/device:CPU:0"

In [4]:
JustTesting = True # If True run everything faster

LoadPreviousModelQ = False # if True we load the model, either for further training or just testing
i_step_load = 360000

RunTrainingQ = False # If True we run the training

RebuildDatasetQ = False # if True we rebuild the dataset
MutateTrainingQ = True # if True, during training we mutate the image
MutateTestingQ = True # if True, during testing we mutate the image

i_step_verify = [] #[240000, 500000]
RunTestQ = True # if True, in the end we test an increasing size of inputs

no_channels = '4TimesClasses' # 'SameClasses' '5PlusClasses' '4TimesClasses'

dataset = 'count_digits'
id_run = 'CA_CD_3Classes_Deeper_M{}_MutTrain{}_MutTest{}'.format(no_channels, MutateTrainingQ, MutateTestingQ) # the prefix to all file names which will be use for saving and loading the model

NO_CLASSES = 3
limits_classes = [2, 8] #len of this should be NO_CLASSES-1
limits_c_p = [0, 2, 8, 100]
H, W = 10, 10

TR_EVOLVE = 50 # Number of time steps to let CA evolve for each input during training
TST_EVOLVE = 50 # Number of time steps to let CA evolve for each input during testing

BATCH_SIZE = 64 # number of images per batch


if no_channels == 'SameClasses':
    NO_CHANNELS = NO_CLASSES # number of hidden states of the CA, must be at least NO_CLASSES because there are two outputs
elif no_channels == '4TimesClasses':
    NO_CHANNELS = 4 * NO_CLASSES # number of hidden states of the CA, must be at least NO_CLASSES because there are two outputs
elif no_channels == '5PlusClasses':
    NO_CHANNELS = 5 + NO_CLASSES # number of hidden states of the CA, must be at least NO_CLASSES because there are two outputs
    
if JustTesting:
    TR_NO_ITERATIONS = 500 # number of iterations for the training loop
    export_every = 250 # number of iterations between each model export
    visualise_every = 50 # number of iteration between each model visualisation
else:
    TR_NO_ITERATIONS = 500000 # number of iterations for the training loop
    export_every = 10000 # number of iterations between each model export
    visualise_every = 2000 # number of iteration between each model visualisation

ADD_NOISE = True # if True then the normal update of the CA has noise added

In [13]:
ca_model.CAModel(NO_CHANNELS, H, W).update_state.summary()

NameError: ignored

In [ ]:
# Prepare the dataset
size_ds = 1e5 #the number of images on the dataset

BuildDS = False
if RebuildDatasetQ:
    BuildDS = True
else:
    try:
        res = np.load(folder + '/dataset/{}_{}.npz'.format(dataset, size_ds))
        x_train = res['x_train']
        y_train = res['y_train']
        x_test = res['x_test']
        y_test = res['y_test']
    except:
        BuildDS = True
if BuildDS:
    x_train, x_test, y_train, y_test = cpd.build_dataset(size_ds, H, W) 
    np.savez(folder + '/dataset/{}_{}.npz'.format(dataset, size_ds), x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

y_train_pic = cpd.to_classes_dim_label(x_train, y_train)
y_label_train = np.zeros(y_train.shape[0])
for i in range(y_train.shape[0]):
    y_label_train[i] = cpd.class_indice_f(y_train[i], limits_classes)
# y_test_pic = to_classes_dim_label(x_test, y_test)

In [ ]:
lr = 1e-3 # initial learning rate
lr_sched = tf.keras.optimizers.schedules.PiecewiseConstantDecay([int(TR_NO_ITERATIONS*0.3), int(TR_NO_ITERATIONS*0.7)], [lr, lr*0.1, lr*0.01])
trainer = tf.keras.optimizers.Adam(lr_sched) # use ADAM optimizer with learning rate schedule

loss_log = np.zeros(TR_NO_ITERATIONS) # for plotting of loss function across time
loss_log_classes = np.zeros((TR_NO_ITERATIONS, NO_CLASSES)) # for plotting of loss function across time

In [ ]:
if LoadPreviousModelQ:
    ca, loss_log, loss_log_classes = get_model(i_step_load)
    ITER = i_step_load
else:
    ca = CAModel()
    ITER = 0

In [ ]:
# Training happens here
if RunTrainingQ:
    # Training Loop
    for i in range(ITER, TR_NO_ITERATIONS):

        b_idx = np.random.randint(0, x_train.shape[0] - 1, size=BATCH_SIZE)
        x0 = ca.initialize(x_train[b_idx])
        y0 = y_train_pic[b_idx]
        y0_label = y_label_train[b_idx]
        y0_label = tf.convert_to_tensor(y0_label)        

        x, loss, c_l = train_step(x0, y0, y0_label)

        loss_log[i] = loss.numpy()
        loss_log_classes[i, :] = [k.numpy() for k in c_l]

        if i % visualise_every == 0:
            clear_output()
            plot_loss(loss_log[:i], loss_log_classes[:i, :], True)
        if i % export_every == 0:
            export_model(ca, i, loss_log, loss_log_classes)

        print('\r step: {}, log10(loss): {}, log10(loss)[classes]: {}'.format(i + 1, np.log10(loss), np.log10(c_l)), end='')
    export_model(ca, TR_NO_ITERATIONS, loss_log, loss_log_classes)

In [ ]:
class VideoWriter:
    def __init__(self, filename, fps=30.0, **kw):
        self.writer = None
        self.params = dict(filename=filename, fps=fps, **kw)

    def add(self, img):
        img = np.asarray(img)
        if self.writer is None:
            h, w = img.shape[:2]
            self.writer = FFMPEG_VideoWriter(size=(w, h), **self.params)
        if img.dtype in [np.float32, np.float64]:
            img = np.uint8(img.clip(0, 1)*255)
        if len(img.shape) == 2:
            img = np.repeat(img[..., None], 3, -1)
        if len(img.shape) == 3 and img.shape[-1] == 4:
            img = img[..., :3] * img[..., 3, None]
        self.writer.write_frame(img)

    def close(self):
        if self.writer:
            self.writer.close()

    def __enter__(self):
        return self

    def __exit__(self, *kw):
        self.close()

def make_run_videos(ca, eval_bs, i_step, seed=1):
    np.random.seed(seed)
    new_idx = np.random.randint(0, x_test.shape[0] - 1, size=eval_bs)
    x = ca.initialize(x_test[new_idx])
    with VideoWriter(folder + '/CA/Movie_model_{}_{}.mp4'.format(id_run, i_step)) as vid:
        for i in tqdm.trange(-1, TST_EVOLVE):
            if MutateTestingQ and i == int(TST_EVOLVE / 2):
                c_i = x[:, :, :, 0]
                x = ca.mutate(x, tf.expand_dims(tf.random.shuffle(c_i), -1))
#             true_images = tf.expand_dims(0.5 - 0 * x[:, :, :, 0], -1)
#             alphas = zoom(tile2d(true_images), scale=4)
#             print(alphas)
            if i == -1:
                image = zoom(tile2d(classify_and_color(ca, x)), scale=4)
            else:
                x = ca(x)
                image = zoom(tile2d(classify_and_color(ca, x)), scale=4)
#             new_image = np.concatenate((image, alphas), axis=-1)
#             print(new_image)
#             im = np.uint8(new_image*255)
            im = np.uint8(image*255)
            im = PIL.Image.fromarray(im)
            draw = PIL.ImageDraw.Draw(im)
            vid.add(np.uint8(im))

In [ ]:
ca

(<__main__.CAModel at 0x7f3647f06e80>,
 array([57.41350555,  7.63882589,  3.03335905, ...,  0.        ,
         0.        ,  0.        ]),
 array([[ 2.22993231, 37.75796127, 92.16165924],
        [ 0.90170944,  6.34681988, 10.06003284],
        [ 0.88766009,  2.69141102,  3.84651518],
        ...,
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]]))

In [ ]:
eval_bs = 5 ** 2 # number of samples in a batch to evaluate
for i_step_v in i_step_verify:
    ca, loss_log, loss_log_classes = get_model(i_step_v)
    make_run_videos(ca, eval_bs, i_step_v)

  0%|          | 1/201 [00:00<00:32,  6.24it/s]

100%|██████████| 201/201 [00:00<00:00, 231.42it/s]


In [ ]:
if RunTestQ:
    def add_pixel(old_image):
        new_image = old_image.copy()
        indices = np.asarray(np.where(old_image == 1)).T
        success = 0
        k = 0
        while (not success) or (k > 100):
            k += 1
            idx = np.random.randint(indices.shape[0])
            x, y = indices[idx, :]
            x_or_y = np.random.rand() < 0.5
            pos_or_neg = np.random.rand() < 0.5
            if x_or_y:
                if pos_or_neg:
                    dx = 1
                    dy = 0
                else:
                    dx = -1
                    dy = 0
            else:
                if pos_or_neg:
                    dx = 0
                    dy = 1
                else:
                    dx = 0
                    dy = -1
            nx = min(max(0, x + dx), 9)
            ny = min(max(0, y + dy), 9)
            if old_image[nx, ny] < 0.5:
                new_image[nx, ny] = 1
                success = 1
        return new_image

    images = np.zeros((20, 25, H, W), dtype=np.float32)
    for i_img in range(25):
        images[0, i_img, int(H / 2), int(W / 2)] = 1
    for j in range(1, 20):
        for i_img in range(25):
            images[j, i_img, :, :] = add_pixel(images[j - 1, i_img, :, :])
    images = tf.constant(images)
    
    for i_step_v in i_step_verify:
        ca, loss_log, loss_log_classes = get_model(i_step_v)
        x = ca.initialize(images[0, :, :, :])
        with VideoWriter(folder + '/CA/Movie_test_increase_{}_{}.mp4'.format(id_run, i_step_v)) as vid:
            for j in range(20):
                x = ca.mutate(x, tf.expand_dims(images[j, :, :, :], -1))
                for i in tqdm.trange(TR_EVOLVE):
                    x = ca(x)
                    image = zoom(tile2d(classify_and_color(ca, x)), scale=4)
                    im = np.uint8(image*255)
                    im = PIL.Image.fromarray(im)
                    draw = PIL.ImageDraw.Draw(im)
                    vid.add(np.uint8(im))
        x = ca.initialize(images[0, :, :, :])
        with VideoWriter(folder + '/CA/Movie_test_decrease_{}_{}.mp4'.format(id_run, i_step_v)) as vid:
            for j in range(20):
                x = ca.mutate(x, tf.expand_dims(images[19 - j, :, :, :], -1))
                for i in tqdm.trange(TR_EVOLVE):
                    x = ca(x)
                    image = zoom(tile2d(classify_and_color(ca, x)), scale=4)
                    im = np.uint8(image*255)
                    im = PIL.Image.fromarray(im)
                    draw = PIL.ImageDraw.Draw(im)
                    vid.add(np.uint8(im))

100%|██████████| 20/20 [00:00<00:00, 295.13it/s]
